Census of Drug and Alcohol Treatment Services in Northern Ireland

In [149]:
from gssutils import *
scraper = Scraper('https://www.health-ni.gov.uk/publications/census-drug-and-alcohol-treatment-services-northern-ireland-2017')
scraper

## Census of drug and alcohol treatment services in Northern Ireland 2017

### Distributions

1. Drug and Alcohol Census 2017 ([application/pdf](https://www.health-ni.gov.uk/sites/default/files/publications/health/drug-alcohol-census-2017.pdf))
1. Data from Census of Drug and Alcohol Treatment Services ([MS Excel Spreadsheet](https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx))
1. Pre-release Access List Drug and Alcohol Census ([application/pdf](https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/pre-release-drug-alcohol-census.pdf))


In [150]:
tabs = {tab.name: tab for tab in scraper.distribution(
    title='Data from Census of Drug and Alcohol Treatment Services').as_databaker()}
tabs.keys()
tabs

{'Contents & Notes to tables': {<B6 'Breakdown by Age and Gender'>, <B12 ''>, <A14 ''>, <B8 'Breakdown by Residential Status'>, <A13 ''>, <A12 ''>, <B10 'Comparison Table'>, <A9 'Table 4'>, <A18 2.0>, <A11 ''>, <B17 'A dashed line (-) represents a cell count of less than three and a * represents a cell that has been masked. This is in order to avoid issues involving personal disclosure, where it may be possible to identify an individual from the data provided.  '>, <B4 ''>, <B9 'Breakdown by Trust'>, <B18 'All percentages are rounded to the nearest percentage point.'>, <A6 'Table 1'>, <A19 3.0>, <B1 ''>, <A10 'Table 5'>, <A4 'Contents'>, <B2 ''>, <B15 ''>, <A3 ''>, <B5 ''>, <A8 'Table 3'>, <A2 '1 March 2017'>, <B16 ''>, <B3 ''>, <A7 'Table 2'>, <B13 ''>, <A15 'Notes to Tables'>, <A17 1.0>, <B19 'Hospital Information System (HIS) figures relate to the number of emergency admissions to HSC hospitals for only drug-related conditions, only alcohol-related conditions or both drug and alcoho

In [151]:
tables = []
for tab_name, script in [
    ('Table 1', 'Treatment Services by Age and Gender.ipynb'),
    ('Table 2', 'Treatment Services by Service Type(Age).ipynb'),
    ('Table 2', 'Treatment Services by Service Type(sex).ipynb'),
    ('Table 3', 'Treatment Services by Residential status(Age).ipynb'),
    ('Table 3', 'Treatment Services by Residential Status(sex).ipynb'),
    ('Table 4', 'Treatment Services by Trust(Age).ipynb'),
    ('Table 4', 'Treatment Services by Trust(sex).ipynb'),
    ('Table 5', 'Treatment Services by Comparison table.ipynb')
]:
    tab = tabs[tab_name]
    %run "$script"
    tables.append(new_table)

In [152]:
tidy = pd.concat(tables)
#tidy.to_csv('testCompare.csv', index = False)

In [153]:
tidy['Treatment Type'].fillna('Total', inplace = True)

In [154]:
#tidy.count()

In [155]:
#tidy['Treatment Type'].unique()

In [156]:
#tidy.head()

In [157]:
#tidy = tidy[tidy['Treatment Type'] == 'Alcohol Only']

In [158]:
#tidy.head()

In [159]:
#tidy['Sex'].unique()
#tidy.to_csv('testCompare.csv', index = False)

In [160]:
tidy.drop(tidy[tidy.Sex.isin(['% of Total', '% of all Males ','% of all Females'])].index, inplace = True)

In [161]:
#tidy['Sex'].unique()

In [162]:
#tidy['Period'].unique()

In [163]:
from datetime import datetime

In [164]:
tidy.Period = pd.to_datetime(tidy.Period).dt.strftime('%Y-%m-%d')

In [165]:
tidy['Period'] = str('day/') + tidy['Period']

In [166]:
#tidy['Age'].unique()
#tidy.to_csv('testCompare.csv', index = False)

In [167]:
tidy['Age'] = tidy['Age'].map(
    lambda x: {
        'Under 18 ' : 'under-18', 
        '18 and over' : '18-plus',
        'All years': 'all' ,
        'All': 'all',
        'All Ages': 'all'
        }.get(x, x))

In [168]:
#tidy['Treatment Type'] = 'alcohol'
#tidy.to_csv('testCompare.csv', index = False)

In [169]:
def user_perc(x,y):
    
    if ( (str(x) == 'Statutory (%)')) | ((str(x) == 'Non-statutory (%)')) | ((str(x) == 'Prison (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Service Type'], row['Measure Type']), axis = 1)



In [170]:
def user_perc(x,y):
    
    if ( (str(x) == 'Residential (%)')) | ((str(x) == 'Non-residential (%)')) | ((str(x) == 'Mixed (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Residential Status'], row['Measure Type']), axis = 1)



In [171]:
#tidy['Health and Social Care Trust'].unique()

In [172]:
def user_perc(x,y):
    
    if ( (str(x) == 'Belfast (%)')) | ((str(x) == 'Northern (%)')) | ((str(x) == 'South Eastern (%)')) | ((str(x) == 'Southern (%)')) | ((str(x) == 'Western (%)')) | ((str(x) == 'Emergency admissions (HIS) (%)')) : 
        
            return 'Percentage'
    else:
            return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Health and Social Care Trust'], row['Measure Type']), axis = 1)

In [173]:
def user_perc(x,y):
    
    if  ((str(x) == 'Emergency admissions (HIS)  (%)')) : 
        
            return 'Percentage'
    else:
            return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Health and Social Care Trust'], row['Measure Type']), axis = 1)

In [174]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].str.rstrip(' (%)')

In [175]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].str.lower()

In [176]:
#tidy['Health and Social Care Trust'].unique()

In [177]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].map(
    lambda x: {
        'south eastern' : 'south-eastern', 
        'emergency admissions (his)' : 'emergency-admissions',
        'emergency admissions (his' : 'emergency-admissions'
        }.get(x, x))

In [178]:
tidy['Measure Type'] = tidy['Measure Type'].map(
    lambda x: {
        'Headcount' : 'Count', 
        'Percentage of Headcount' : 'Percentage',
        }.get(x, x))

In [179]:
#tidy['Sex'].unique()

In [180]:
def user_perc(x,y):
    
    if ( (str(x) == 'Male (%)')) | ((str(x) == 'Female (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Sex'], row['Measure Type']), axis = 1)



In [181]:
tidy['Sex'] = tidy['Sex'].map(
    lambda x: {
        'Female' : 'F', 
        'Male' : 'M',
        'Persons' : 'T',
        'Female  ' : 'F',
        'Male (%)' : 'M',
        'Female (%)': 'F'
        }.get(x, x))

In [182]:
tidy['Sex'] = tidy['Sex'].str.rstrip('(%)')

In [183]:
tidy['Service Type'].unique()

array(['all', 'Statutory', 'Non-statutory', 'Prison', 'Statutory (%)',
       'Non-statutory (%)', 'Prison (%)', 'All'], dtype=object)

In [184]:
tidy['Service Type'] = tidy['Service Type'].map(
    lambda x: {
        'Total' : 'all',
        'total' : 'all'
        }.get(x, x))

In [185]:
tidy['Service Type'] = tidy['Service Type'].str.rstrip(' (%)')

In [186]:
tidy['Service Type'] = tidy['Service Type'].str.lower()

In [187]:
tidy['Residential Status'] = tidy['Residential Status'].str.rstrip(' (%)')

In [188]:
tidy['Residential Status'] = tidy['Residential Status'].str.lower()

In [189]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].map(
    lambda x: {
        'total' : 'all' 
        }.get(x, x))

In [190]:
tidy['Residential Status'] = tidy['Residential Status'].map(
    lambda x: {
        'total' : 'all' 
        }.get(x, x))

In [191]:
#tidy.head()

In [192]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
tidy.to_csv(out / 'observations.csv', index = False)
#tidy.to_csv('testCompare.csv', index = False)
#tidy['Treatment Type'].unique()

In [193]:
tidy[tidy['Measure Type'] != 'Percentage'].to_csv(out / 'observations-no-percentages.csv', index=False)

There's a metadata tab in the spreadsheet with abstract and contact details. **Todo: extract these and also figure out what the license should really be.**

In [194]:
md_tab = tabs['Metadata']
md_heading = md_tab.filter('Metadata')
md_heading.assert_one()
properties = md_heading.fill(DOWN).is_not_whitespace()
values = properties.fill(RIGHT).is_not_whitespace()
headings = properties - values.fill(LEFT)
properties = properties - headings
cs = ConversionSegment(values, [
    HDim(properties, "Property", DIRECTLY, LEFT),
    HDim(headings, "Heading", CLOSEST, UP)
], includecellxy=True)
#savepreviewhtml(cs)

In [195]:
from gssutils.metadata import THEME

for i, row in cs.topandas().iterrows():
    v = md_tab.get_at(row.__x, row.__y).value.strip()
    if row.Property == 'Abstract':
        scraper.dataset.description = v
    elif row.Property == 'National Statistics Theme':
        scraper.dataset.theme = THEME[pathify(v)]
    elif row.Property == 'Keyword':
        scraper.dataset.keyword = map(lambda x: x.strip(), v.split(' '))
    elif row.Property == 'Email Address':
        scraper.dataset.contactEmail = f'mailto:{v}'

In [196]:
scraper.dataset.family = 'health'
scraper.dataset.license = 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'

with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

In [197]:
tidy.to_csv('testCompare.csv', index = False)